### Notes
- what driving receipt of statement
- only need a human when someone picks up the phone


#### Random Forest Requests on target Status
- CallerStatus: 1
- CustomerName: 2
- RequestType: 3
- RequesterFullName: 4
- CurrentAssigneeName: 5
- Contact: 6
- RequestMethod: 7
- Volume: 8

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .appName('PySparkShell')
    .getOrCreate()
)

sc = spark.sparkContext
# sc.defaultParallelism

# # spark.conf.set('spark.sql.session.timeZone', 'UTC')
# # spark.conf.set('spark.sql.session.timeZone', 'America/New_York')

In [2]:
import sys
import os

sys.path.append('/dbfs/cdr/library')

from pandas import set_option
set_option('display.max.columns', None)

# TODO: remove for Databricks
from dbutils import DBUtils, FileInfo
dbutils = DBUtils()

In [3]:
from sparkFunctions import create_tmp_parquet

from typing import Dict, List, Optional

# adds file system prefix /dbfs for local and dbfs: for databricks
def resolve(path: str) -> Optional[str]:
    try:
        prefix = dbutils.fs.ls('/')[0].path[:5]
        return prefix + path
    except Exception as error:
        print(error)

In [4]:
from pyspark.sql import Column, DataFrame
from pyspark.sql.functions import approx_count_distinct, collect_set, date_trunc, desc, size, trim

from pyspark.storagelevel import StorageLevel

from typing import Dict, List

def truncateTimestamp(dataFrame: DataFrame) -> DataFrame:
    for column, dtype in dataFrame.dtypes:
        if dtype == 'timestamp':
            dataFrame = dataFrame.withColumn(column, date_trunc('second', column))
        
    return dataFrame


def trimString(dataFrame: DataFrame) -> DataFrame:
    for column, dtype in dataFrame.dtypes:
        if dtype == 'string':
            dataFrame = dataFrame.withColumn(column, trim(column))
        
    return dataFrame



def distinctValuesMap(
    dataFrame: DataFrame,
    keys: List[str],
    debug: bool=False
) -> Dict[str, int]:
    
    valuesMap = dict()
    
    for column in dataFrame.columns:
        if column not in keys:
            values = (
                dataFrame
                .groupBy(keys)
                .agg(approx_count_distinct(column).alias('count'))
                .where('count > 1')
                .count()
            )
            
            if debug:
                print(column, values)
            
            if values > 1:
                valuesMap[column] = values
    
    return valuesMap


def columnFactorCounts(
    dataFrame: DataFrame,
    keys: List[str],
    column: str
) -> DataFrame:
    
    return (
        dataFrame
        .groupBy(keys)
        .agg(approx_count_distinct(column).alias(column))
        .sort(desc(column))
    )


def columnFactorArray(
    dataFrame: DataFrame,
    keys: List[str],
    column: str
) -> DataFrame:
    
    return (
        dataFrame
        .groupBy(keys)
        .agg(collect_set(column).alias(column))
        .withColumn('size', size(column))
        .sort(desc('size'))
    )

In [5]:
FOLDER = '/automation'

ZOOM_FOLDER = os.path.join(FOLDER, 'ZoomData')
BASE_FOLDER = os.path.join(FOLDER, 'OnBase')

In [23]:
dbutils.fs.ls(BASE_FOLDER)

[FileInfo(path='/dbfs/automation/OnBase/cc_STStatementEmailDocs.parquet', name='cc_STStatementEmailDocs.parquet', size=25563408),
 FileInfo(path='/dbfs/automation/OnBase/rm_DVStatementRequestActivityRecords.parquet', name='rm_DVStatementRequestActivityRecords.parquet', size=97480388),
 FileInfo(path='/dbfs/automation/OnBase/rm_DVStatementRequests.parquet', name='rm_DVStatementRequests.parquet', size=51427694),
 FileInfo(path='/dbfs/automation/OnBase/rm_DVStatements.parquet', name='rm_DVStatements.parquet', size=40973701),
 FileInfo(path='/dbfs/automation/OnBase/rm_DVVendorContacts.parquet', name='rm_DVVendorContacts.parquet', size=24426383)]

In [9]:
from pyspark.sql.functions import col 

path = os.path.join(ZOOM_FOLDER, 'formattedAccountCallLogs.parquet')
calls = spark.read.parquet(resolve(path))


# Eastern Standard Time
calls = calls.withColumn('date_time', col('date_time_est').cast('timestamp')).drop('date_time_est')

integers = [
    'duration',
    'extension_number',
    'caller_number_type',
    'callee_number_type'
]

for column in integers:
    calls = calls.withColumn(column, col(column).cast('integer'))

calls = calls.withColumn('call_id', col('call_id').cast('long'))

drops = [
    'id',
    'path',
    'has_recording'
]

calls = calls.drop(*drops)

calls = truncateTimestamp(calls)
calls = trimString(calls)

In [10]:
%%time
from pyspark.sql.functions import regexp_extract

# outgoing calls begin with +1
# also use 1 for 1-800 numbers
# ones not found in VENDOR table

# ^ is anchor at start
# \+ escapes literal + sign in regular expression
# same with *, which is generally a wild card
pattern = r'^(\+1|\*)*([0-9]+)'

calls = (
    calls
    # extract second group using index=2
    .withColumn('CALLED_NUMBER', regexp_extract('callee_number', pattern, 2))
    .withColumn('RECEIVED_NUMBER', regexp_extract('caller_number', pattern, 2))
)

# calls = create_tmp_parquet(spark, calls, '/tmp/CALLS')

calls.persist(StorageLevel.DISK_ONLY)

calls.limit(5).toPandas()

CPU times: user 73.3 ms, sys: 49.7 ms, total: 123 ms
Wall time: 4.11 s


,caller_name,caller_number,callee_name,callee_number,direction,duration,date_time,call_end_time,result,call_id,extension_number,name,type,call_type,caller_number_type,callee_number_type,has_voicemail,date_time_end_est,CALLED_NUMBER,RECEIVED_NUMBER
0,None,+16169702069,Mitchell Malling,442,inbound,86,2021-09-17 11:05:47,None,Call connected,7008919107000321976,442,Mitchell Malling,user,voip,2,1,False,None,442,6169702069
1,Yodit Kahssai,474,None,+18552675551,outbound,139,2021-09-17 11:06:06,2021-09-17T15:08:27Z,Call connected,7008919188613195707,474,Yodit Kahssai,user,pstn,1,2,False,2021-09-17 11:08:27.000,8552675551,474
2,None,+16169702069,Nathan Gregorio,270,inbound,14,2021-09-17 11:07:00,None,Call connected,7008919416247491134,270,Nathan Gregorio,user,voip,2,1,False,None,270,6169702069
3,Dornier Medtech,+17705146253,Main Auto Receptionist,901,inbound,0,2021-09-17 11:07:03,None,No Answer,7008919433427273168,901,Main Auto Receptionist,autoReceptionist,voip,2,1,False,None,901,7705146253
4,Xavier Baron,852,Vssp,+16147195221,outbound,31,2021-09-17 11:07:07,2021-09-17T15:07:49Z,Call connected,7008919450606123974,852,Xavier Baron,user,pstn,1,2,False,2021-09-17 11:07:49.000,6147195221,852


In [7]:
%%time
# from pyspark.sql.functions import col

path = os.path.join(BASE_FOLDER, 'rm_DVStatementRequestActivityRecords.parquet')
activities = spark.read.parquet(resolve(path))

dates = [
    'ActivityDate',
    'FollowUpDate'
]

for column in dates:
    activities = activities.withColumn(column, col(column).cast('date'))

# drops = [
#     'STNID',
#     'CustVendorObjectID'
# ]

# activities = activities.drop(*drops)
    
activities = truncateTimestamp(activities)
activities = trimString(activities)
    
# activities = create_tmp_parquet(spark, activities, '/tmp/ACTIVITIES')

activities.persist(StorageLevel.DISK_ONLY)
    
activities.limit(5).toPandas()

CPU times: user 73.8 ms, sys: 28.7 ms, total: 102 ms
Wall time: 25.2 s


,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,28785475,2020-11-02 06:41:33,1279250,ULINE,3412,Carolinas Shared Service - 3412,Call,ASIBLEY,2020-11-01,2020-11-03,NaN,This is a test - Updated,None,None,21259327,NaN,25422522,ASIBLEY
1,28785637,2020-11-02 06:47:27,1279250,ULINE,3412,Carolinas Shared Service - 3412,Email,ASIBLEY,2020-11-01,2020-11-04,NaN,Test 2 - Updated,None,None,21259327,NaN,25422522,ASIBLEY
2,28786094,2020-11-02 06:52:58,1279250,ULINE,3412,Carolinas Shared Service - 3412,Client Email,ASIBLEY,2020-11-01,2020-11-03,NaN,Test 3,None,None,21259327,NaN,25422522,ASIBLEY
3,28838136,2020-11-04 12:47:58,1287154,BIOCOMPOSITES INC.,3435,The Christ Hospital - 3435,None,bwilliams1,2020-11-03,None,6192802.0,Statement Request creation skipped because thi...,None,None,27542411,NaN,27781594,ASIBLEY
4,28838138,2020-11-04 12:48:02,1225130,BONA FIDE COMMERCIAL SERVICES,3177,UC Health - 3177,None,jdagher,2020-11-03,None,6192806.0,Statement Request creation skipped because thi...,None,None,24317597,NaN,25860124,ASIBLEY


In [8]:
%%time
path = os.path.join(BASE_FOLDER, 'rm_DVStatementRequests.parquet')
requests = spark.read.parquet(resolve(path))

dates = [
    'RequestDate',
    'LastActivityDate'
]

for column in dates:
    requests = requests.withColumn(column, col(column).cast('date'))

requests = requests.withColumn('Volume', col('Volume').cast('double'))

# drops = [
#     'MessageID',
#     'VendorContactObjectID'
# ]

# requests = requests.drop(*drops)

requests = truncateTimestamp(requests)    
requests = trimString(requests)

requests.persist(StorageLevel.DISK_ONLY)

requests.limit(5).toPandas()

CPU times: user 79.7 ms, sys: 10.9 ms, total: 90.5 ms
Wall time: 11.7 s


,JobNo,JobTier,CustomerName,CustVendorID,VendorNo,CustVendorGroupID,WNC,StatementWNC,VendorGroupName,Volume,RequestDate,ReferenceNumber,Status,RequestMethod,RequestType,Contact,RequesterFullName,RequestText,LastActivityDate,LastStatementReceivedDate,CreatedDate,CallsheetNo,ObjectID,CallerStatus,ReconStatus,CurrentAssigneeID,CurrentAssigneeName,EnteredReconDate,LastReconQueueName,LastReconQueueEntryDate,AccountsReceived,AccountsRequested,MessageID,VendorContactObjectID,WebsiteVendor,WNCSpecialHandling,NeedLeadVendor
0,3427,NaN,New Hanover Regional Medical Center,415960,16845,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC,0.0,2021-09-02,1285660,No Receipt,MassEmail,Caller,amcoffeedist@gmail.com,Cindy Allen,None,2021-08-22,NaT,2021-09-02 22:50:28,C-338914,36846273,Sent Authorization Letter,None,19272454,Yodit Kahssai,NaT,None,NaT,NaN,NaN,hONrG5xpTUyiQB4Rs7gb0g,29510277.0,NaN,NaN,NaN
1,3427,NaN,New Hanover Regional Medical Center,415960,16845,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC,0.0,2020-07-23,1285660,Superceded,MassEmail,None,amcoffeedist@gmail.com,Jessica Rinehart,None,2020-07-23,NaT,2020-07-24 12:29:52,C-338914,26681846,None,None,19272454,Yodit Kahssai,NaT,None,NaT,NaN,NaN,None,NaN,NaN,NaN,NaN
2,3427,NaN,New Hanover Regional Medical Center,415960,16845,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC,0.0,2020-07-30,1285660,Superceded,MassEmail,Mass 1,amcoffeedist@gmail.com,Jessica Rinehart,None,2020-07-30,NaT,2020-07-31 04:05:15,C-338914,26819776,None,None,19272454,Yodit Kahssai,NaT,None,NaT,NaN,NaN,None,NaN,NaN,NaN,NaN
3,3427,NaN,New Hanover Regional Medical Center,415960,16845,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC,0.0,2021-01-18,1285660,Superceded,MassEmail,Caller,amcoffeedist@gmail.com,Jessica Rinehart,None,2020-07-30,NaT,2021-01-19 07:01:21,C-338914,30690711,Sent Authorization Letter,None,8398263,Rebekah Dykema,NaT,None,NaT,NaN,NaN,None,29510277.0,NaN,NaN,NaN
4,3427,NaN,New Hanover Regional Medical Center,415960,16845,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC,0.0,2021-08-18,1285660,Superceded,MassEmail,Mass 1,amcoffeedist@gmail.com,Cindy Allen,None,2021-08-22,NaT,2021-08-22 20:03:52,C-338914,36365526,Sent Authorization Letter,None,8398263,Rebekah Dykema,NaT,None,NaT,NaN,NaN,hONrG5xpTUyiQB4Rs7gb0g,29510277.0,NaN,NaN,NaN


In [16]:
%%time
from pandas import read_parquet

# requests.count() # 346,199
# requests.where('MessageID IS NOT NULL').count() # 84,709
# requests.where('MessageID IS NOT NULL').select('MessageID').dropDuplicates().count() # 42,473

# requests.write.parquet('/tmp/REQUESTS', mode='overwrite')

# table = read_parquet('/tmp/REQUESTS')

# table.head()

# table.to_parquet('~/Desktop/REQUESTS.parquet')

# table = read_parquet('~/Desktop/REQUESTS.parquet')

# columns = [
#     'CallerStatus',
#     'CustomerName',
#     'RequestType',
#     'RequesterFullName',
#     'CurrentAssigneeName',
#     'Contact',
#     'RequestMethod',
#     'Volume',
#     'Status'
# ]

# table[columns].fillna('NA').to_csv('~/Desktop/REQUESTS.csv', index=False)

CPU times: user 2.54 s, sys: 135 ms, total: 2.68 s
Wall time: 3.07 s


In [20]:
%%time
path = os.path.join(BASE_FOLDER, 'rm_DVStatements.parquet')

statements = spark.read.parquet(resolve(path))

statements = truncateTimestamp(statements)
statements = trimString(statements)

statements.persist(StorageLevel.DISK_ONLY)

statements.limit(5).toPandas()

CPU times: user 52.8 ms, sys: 7.69 ms, total: 60.4 ms
Wall time: 9.44 s


,ReferenceNumber,JobNo,JobTier,StatementDate,DocumentHandle,ObjectID,CustVendorObjID,CustVendGroupObjID,CustVendGroupName,CustomerName,CID,CustVendName,CustVendNo,Volume,AccountsIdentified,Recon,EmailMessageID,CreatedDate,SRARObjectId
0,1074630,2600,NaN,2019-08-08 20:00:00,NaN,20482171,19080046,19080045,TEST - HILL-ROM COMPANY INC,"BSI Healthcare Audit Services, LLC",15121,TEST - HILL-ROM COMPANY INC,2340,2014673.00,NaN,NaN,0000000085C36CE5BB5D9244B44E28C87A7C112B070049...,2019-08-09 09:38:05,NaN
1,1074630,2600,NaN,2019-08-08 20:00:00,NaN,20482173,19080046,19080045,TEST - HILL-ROM COMPANY INC,"BSI Healthcare Audit Services, LLC",15121,TEST - HILL-ROM COMPANY INC,2340,2014673.00,NaN,NaN,0000000085C36CE5BB5D9244B44E28C87A7C112B070049...,2019-08-09 09:45:07,NaN
2,1074630,2600,NaN,2019-08-08 20:00:00,NaN,20482385,19080046,19080045,TEST - HILL-ROM COMPANY INC,"BSI Healthcare Audit Services, LLC",15121,TEST - HILL-ROM COMPANY INC,2340,2014673.00,NaN,NaN,0000000085C36CE5BB5D9244B44E28C87A7C112B070049...,2019-08-09 12:06:44,NaN
3,1074630,2600,NaN,2019-08-08 20:00:00,NaN,20482388,19080046,19080045,TEST - HILL-ROM COMPANY INC,"BSI Healthcare Audit Services, LLC",15121,TEST - HILL-ROM COMPANY INC,2340,2014673.00,NaN,NaN,0000000085C36CE5BB5D9244B44E28C87A7C112B070049...,2019-08-09 12:09:29,NaN
4,1157916,2936,1.0,2019-08-13 20:00:00,NaN,20512575,6523144,6523143,AMBU INC,Providence,12146,AMBU INC,25171,713229.48,NaN,NaN,00000000F643DA057DBD124B829A30CFCF400C1507002B...,2019-08-14 05:37:44,NaN


In [25]:
%%time
path = os.path.join(BASE_FOLDER, 'cc_STStatementEmailDocs.parquet')

emails = spark.read.parquet(resolve(path))

emails = truncateTimestamp(emails)
emails = trimString(emails)

emails.persist(StorageLevel.DISK_ONLY)

emails.limit(5).toPandas()

CPU times: user 51.6 ms, sys: 7.54 ms, total: 59.2 ms
Wall time: 5.69 s


,DocumentHandle,DateCreated,MAILDateTime,MAILFromAddress,MAILToAddress,MAILCcAddress,MAILSubject,MAILMessageID,MAILAttachmentCount,S-Ref#,S-CustomerName,S-Job#,S-VendorName,S-CreatedByUser,S-SkipAutoReceive,S-Recon,S-LargeCredits,IngestionSource,S-StatementExistsInBatch
0,15594200,2019-06-12 05:22:11,2019-06-07 15:14:29,None,None,None,RE: STATEMENT REQUEST - FAIRVIEW HEALTH SERVIC...,00000000F643DA057DBD124B829A30CFCF400C1507002B...,11,1230333,None,NaN,None,None,None,None,None,None,None
1,15595567,2019-06-12 05:28:05,2019-06-07 15:14:17,None,None,None,RE: STATEMENT REQUEST (REF # 1190860),00000000F643DA057DBD124B829A30CFCF400C1507002B...,7,1190860,None,NaN,None,None,None,None,None,None,None
2,15595581,2019-06-12 05:40:03,2019-06-07 15:12:18,None,None,None,RE: INFORMATION REQUEST #1237232,00000000F643DA057DBD124B829A30CFCF400C1507002B...,2,1237232,None,NaN,None,None,None,None,None,None,None
3,15595588,2019-06-12 05:42:47,2019-06-07 15:15:58,None,None,None,REF#1157275 STATEMENT FROM PROVATION MEDICAL F...,00000000F643DA057DBD124B829A30CFCF400C1507002B...,2,1157275,None,NaN,None,None,None,None,None,None,None
4,15595595,2019-06-12 05:49:49,2019-06-07 15:15:37,None,None,None,RE: INFORMATION REQUEST #1217277,00000000F643DA057DBD124B829A30CFCF400C1507002B...,2,1217277,None,NaN,None,None,None,None,None,None,None


In [28]:
from sparkFunctions import countDuplicates

# countDuplicates(emails, ['DocumentHandle']).count() # 0
# countDuplicates(emails, ['MAILMessageID']).count() # 593

In [32]:
# %%time
# activity = (
#     activities
#     .join(
#         requests,
#         on=activities['StatementRequestObjectID'] == requests['ObjectID'],
#         how='left_semi'
#     )
# )

# activity.persist(StorageLevel.DISK_ONLY).count()

activity.limit(5).toPandas()

,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,28785475,2020-11-02 06:41:33,1279250,ULINE,3412,Carolinas Shared Service - 3412,Call,ASIBLEY,2020-11-01,2020-11-03,NaN,This is a test - Updated,None,None,21259327,NaN,25422522,ASIBLEY
1,28785637,2020-11-02 06:47:27,1279250,ULINE,3412,Carolinas Shared Service - 3412,Email,ASIBLEY,2020-11-01,2020-11-04,NaN,Test 2 - Updated,None,None,21259327,NaN,25422522,ASIBLEY
2,28786094,2020-11-02 06:52:58,1279250,ULINE,3412,Carolinas Shared Service - 3412,Client Email,ASIBLEY,2020-11-01,2020-11-03,NaN,Test 3,None,None,21259327,NaN,25422522,ASIBLEY
3,28838136,2020-11-04 12:47:58,1287154,BIOCOMPOSITES INC.,3435,The Christ Hospital - 3435,None,bwilliams1,2020-11-03,None,6192802.0,Statement Request creation skipped because thi...,None,None,27542411,NaN,27781594,ASIBLEY
4,28838138,2020-11-04 12:48:02,1225130,BONA FIDE COMMERCIAL SERVICES,3177,UC Health - 3177,None,jdagher,2020-11-03,None,6192806.0,Statement Request creation skipped because thi...,None,None,24317597,NaN,25860124,ASIBLEY


In [35]:
%%time
# each StatementRequestObjectID maps to one Customer/Vendor, Job, and ReferenceNumber
# FollowUpDate mostly null
valuesMap = distinctValuesMap(activity, ['StatementRequestObjectID'], debug=True)

ObjectID 142537
CreatedDate 142529
ReferenceNumber 0
CustomerVendorName 0
JobNumber 0
JobName 0
ContactType 62554
ActivityUser 122683
ActivityDate 109358
FollowUpDate 1
STNID 37376
Notes 123734
Outcome 79639
ActivityType 8986
CustVendorObjectID 10
VendorContactObjectID 3953
CreatedBy 42491
CPU times: user 91.9 ms, sys: 25 ms, total: 117 ms
Wall time: 1min 2s


In [138]:
request = (
    requests
    .join(
        activity,
        on=[
            activity['StatementRequestObjectID'] == requests['ObjectID'],
            # activity['Outcome'] == requests['CallerStatus']
        ],
        how='left_semi'
    )
)

request.persist(StorageLevel.DISK_ONLY).count()

260289

In [146]:
# request.createOrReplaceTempView(name='request')
# activity.createOrReplaceTempView(name='activity')

# not all activity has Outcome filled with value to update the request caller status
(
    request
    .join(
        activity,
        on=[
            activity['StatementRequestObjectID'] == request['ObjectID'],
            activity['Outcome'].eqNullSafe(request['CallerStatus'])
        ],
        how='left_anti'
    )
    .toPandas()
)

,JobNo,JobTier,CustomerName,CustVendorID,VendorNo,CustVendorGroupID,WNC,StatementWNC,VendorGroupName,Volume,RequestDate,ReferenceNumber,Status,RequestMethod,RequestType,Contact,RequesterFullName,RequestText,LastActivityDate,LastStatementReceivedDate,CreatedDate,CallsheetNo,ObjectID,CallerStatus,ReconStatus,CurrentAssigneeID,CurrentAssigneeName,EnteredReconDate,LastReconQueueName,LastReconQueueEntryDate,AccountsReceived,AccountsRequested,MessageID,VendorContactObjectID,WebsiteVendor,WNCSpecialHandling,NeedLeadVendor
0,3177,2.0,UC Health,18169231,100063533,18169230,0,NaN,HENRY SCHEIN INC - DENTAL,262562.01,2020-08-13,1225093,Fully Received,MassEmail,Mass 1,Megan.Albinowski@henryschein.com,Cindy Allen,None,2020-11-15,2020-11-16 19:00:00,2020-08-14 06:05:32,C-295396,27434056,Statement Received,None,23668664,Josephine Dagher,NaT,None,NaT,NaN,NaN,None,NaN,NaN,NaN,NaN
1,3306,NaN,Montefiore Health System,21988911,117635,21988910,0,NaN,SIRTEX MEDICAL INC.,0.00,2020-08-16,1259437,Fully Received,MassEmail,Mass 1,bryant.gamez@sirtex.com,Bill Wilhoit,None,2020-11-18,2020-11-19 19:00:00,2020-08-17 06:44:48,C-303935,27442628,Statement Received,None,8406307,Carla Roark,NaT,None,NaT,NaN,NaN,None,NaN,NaN,NaN,NaN
2,3431,NaN,San Antonio Regional Hospital,27645383,0000001501,27645381,0,NaN,PASADENA BAKING CO,66456.58,2020-09-02,1287721,Fully Received,MassFax,Mass 1,16267960081@efaxsend.com,Jessica Rinehart,"<!DOCTYPE html><html><BASE href=""http://efax.c...",2020-12-02,2020-12-02 19:00:00,2020-09-03 04:08:58,C-348725,27757104,Statement Received,None,8405658,Felicia Bishop,NaT,None,NaT,NaN,NaN,None,29605093.0,0.0,NaN,NaN
3,2775,1.0,BJC Healthcare,22523106,0000005028,22523105,0,NaN,ARTEC ENVIRONMENTAL MONITORING,248662.57,2020-09-10,1049623,Fully Received,MassEmail,Mass 1,deanna@artecenvironmental.com,Bill Wilhoit,None,2020-11-12,2020-11-17 19:00:00,2020-09-11 04:48:14,C-305127,27852339,Statement Received,None,20420366,Christine Brooks,NaT,None,NaT,NaN,NaN,None,NaN,NaN,NaN,NaN
4,3212,NaN,Fairview Health Services,15316433,0000165185,15316432,0,NaN,DELL SOFTWARE INC,11812.32,2020-10-11,1230227,Fully Received,MassEmail,Mass 1,accountsreceivable@quest.com,Bill Wilhoit,None,2020-11-15,2020-11-30 19:00:00,2020-10-12 05:12:30,C-315977,28462804,Statement Received,None,19272454,Yodit Kahssai,NaT,None,NaT,NaN,NaN,None,31410727.0,NaN,NaN,NaN


In [150]:
# activities.where('StatementRequestObjectID = 27442628').toPandas()

In [52]:
# columnFactorCounts(activity, ['StatementRequestObjectID'], 'ContactType').show()

# activity.groupBy('FollowUpDate').count().show()

# activity.where("ContactType = 'N/A'").count()

# activity.groupBy('ContactType').count().show()

columnFactorArray(activity, ['StatementRequestObjectID'], 'ContactType').limit(10).toPandas()

,StatementRequestObjectID,ContactType,size
0,30633982,"[N/A, Email, Call, Client Email]",4
1,33476087,"[N/A, Email, Call, Client Email]",4
2,29447834,"[N/A, Call, Fax, Client Email]",4
3,31947117,"[N/A, Email, Call, Client Email]",4
4,28887815,"[N/A, Email, Call, Client Email]",4
5,30784634,"[N/A, Email, Call, Fax]",4
6,34050529,"[N/A, Email, Call, Client Email]",4
7,28887824,"[N/A, Email, Call, Client Email]",4
8,30634501,"[N/A, Email, Call, Client Email]",4
9,31195395,"[N/A, Email, Call, Client Email]",4


In [57]:
keys = [
    'JobNumber',
    'CustomerVendorName',
    'ReferenceNumber'
]

columnFactorArray(activity, keys, 'ContactType').limit(10).toPandas()

,JobNumber,CustomerVendorName,ReferenceNumber,ContactType,size
0,3100,COSMAN MEDICAL INC,1222378,"[N/A, Email, Call, Fax, Client Email]",5
1,3065,PHONAK LLC,1218252,"[N/A, Email, Call, Fax, Client Email]",5
2,2775,FISHER HEALTHCARE,1049287,"[N/A, Email, Call, Fax, Client Email]",5
3,3435,GE PRECISION HEALTHCARE LLC,1286907,"[N/A, Email, Call, Fax, Client Email]",5
4,3354,CHANGE HEALTHCARE,1274698,"[N/A, Email, Call, Fax, Client Email]",5
5,3444,PENOBSCOT RESPIRATORY P A,1293399,"[N/A, Email, Call, Fax, Client Email]",5
6,3100,AWI FIXTURES AND INTERIORS INC,1222741,"[N/A, Email, Call, Fax, Client Email]",5
7,3022,GE MEDICAL SYSTEMS INFORMATION TECHNOLOGIES,1191589,"[N/A, Email, Call, Fax, Client Email]",5
8,3285,SANOFI PASTEUR INCORPORATED,1239311,"[N/A, Email, Call, Fax, Client Email]",5
9,3063,LABORIE MED TECHNOLOGIES CORP,1277182,"[N/A, Email, Call, Fax, Client Email]",5


In [73]:
conditions = [
    "JobNumber = 3100",
    "CustomerVendorName = 'COSMAN MEDICAL INC'",
    "ReferenceNumber = 1222378"
]

condition = ' AND '.join(conditions)

activity.where(condition).sort('CreatedDate').toPandas()

,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,29327321,2020-11-30 06:32:00,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,Call,bwilliams1,2020-11-29,None,6247726.0,Sent email request.,None,None,25435716,NaN,29327320,SVC_WORKFLOW
1,31266525,2021-02-10 11:45:03,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,Email,Cindy Allen,2021-02-09,None,NaN,None,Sent Authorization Letter,None,25435716,NaN,31266524,SVC_WORKFLOW
2,31830335,2021-02-25 06:59:45,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,Call,Becky Williams,2021-02-24,None,6509981.0,Unable to complete call at this time. Record...,Need to Receive,None,25435716,NaN,31821304,SVC_WORKFLOW
3,31852812,2021-02-26 03:39:05,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,Email,Bill Wilhoit,2021-02-25,None,NaN,None,Sent Authorization Letter,None,25435716,31452524.0,31852811,SVC_WORKFLOW
4,32105474,2021-03-10 07:11:19,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,Email,Cindy Allen,2021-03-09,None,NaN,None,Sent Authorization Letter,None,25435716,31452524.0,32105472,SVC_WORKFLOW
5,32274948,2021-03-19 05:02:12,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,None,Becky Williams,2021-03-18,None,NaN,Sent an email request.,Sent Authorization Letter,Called Vendor,25435716,31452524.0,32271850,BWILLIAMS1
6,32496521,2021-03-30 04:50:23,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,None,Becky Williams,2021-03-29,None,NaN,Unable to complete call at this time. Record...,Left Voicemail,Called Vendor,25435716,31452524.0,32271850,BWILLIAMS1
7,32899270,2021-04-13 05:16:44,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,None,Becky Williams,2021-04-12,None,NaN,Sent email request to Accounting.,Sent Authorization Letter,Called Vendor,25435716,31452524.0,32271850,BWILLIAMS1
8,33170067,2021-04-23 09:32:55,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,None,Becky Williams,2021-04-22,None,NaN,Faxed request.,Will Email or Fax,Called Vendor,25435716,31452524.0,32271850,BWILLIAMS1
9,33478666,2021-05-06 12:40:06,1222378,COSMAN MEDICAL INC,3100,VCU Health Systems - 3100,N/A,Billy Ong,2021-05-05,None,6682461.0,Will send request via client email tomorrow,Will Email or Fax,None,25435716,NaN,32271850,SVC_WORKFLOW


In [74]:
from pyspark.sql.functions import col

key = 'StatementRequestObjectID'
value = 30633982

activity.where(col(key) == value).sort('CreatedDate').toPandas()

,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,30936765,2021-01-25 09:18:09,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,Call,Yodit Kahssai,2021-01-24,None,6427431.0,sent an email,Sent Authorization Letter,None,21993184,NaN,30633982,SVC_WORKFLOW
1,31090582,2021-02-01 09:10:57,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,Call,Yodit Kahssai,2021-01-31,None,6446658.0,lvm for AR,Left Voicemail,None,21993184,NaN,30633982,SVC_WORKFLOW
2,31122654,2021-02-03 03:44:41,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,Client Email,Dorthy Kuester,2021-02-02,None,6451230.0,emailed CerapedicsAccounts@cerapedics.com remo...,Left Voicemail,None,21993184,NaN,30633982,SVC_WORKFLOW
3,31122656,2021-02-03 03:44:43,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,Client Email,Dorthy Kuester,2021-02-02,None,6451232.0,emailed CerapedicsAccounts@cerapedics.com remo...,Left Voicemail,None,21993184,NaN,30633982,SVC_WORKFLOW
4,31341318,2021-02-15 05:48:11,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,N/A,Tammie Miller,2021-02-14,None,6479513.0,"removed ph#(303) 974-6275, fax#(888) 671-4277,...",Statement Received,None,21993184,NaN,30633982,SVC_WORKFLOW
5,32303854,2021-03-22 04:17:44,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,None,Avreil Markham,2021-03-21,None,NaN,None,None,None,21993184,31405165.0,30633982,AMARKHAM
6,32303904,2021-03-22 04:26:08,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,None,Avreil Markham,2021-03-21,None,NaN,None,None,None,21993184,31405165.0,30633982,AMARKHAM
7,32304067,2021-03-22 04:38:54,1238291,CERAPEDICS INC,3285,Community Health Systems - 3285,Email,Avreil Markham,2021-03-21,None,NaN,"Emailed Jason, jbell@cerapedics.com regarding ...",None,None,21993184,31405165.0,30633982,AMARKHAM


In [75]:
key = 'ObjectID'
value = 30633982

requests.where(col(key) == value).sort('CreatedDate').toPandas()

,JobNo,JobTier,CustomerName,CustVendorID,VendorNo,CustVendorGroupID,WNC,StatementWNC,VendorGroupName,Volume,RequestDate,ReferenceNumber,Status,RequestMethod,RequestType,Contact,RequesterFullName,RequestText,LastActivityDate,LastStatementReceivedDate,CreatedDate,CallsheetNo,ObjectID,CallerStatus,ReconStatus,CurrentAssigneeID,CurrentAssigneeName,EnteredReconDate,LastReconQueueName,LastReconQueueEntryDate,AccountsReceived,AccountsRequested,MessageID,VendorContactObjectID,WebsiteVendor,WNCSpecialHandling,NeedLeadVendor
0,3285,NaN,Community Health Systems,21993184,1011142,21993183,0,NaN,CERAPEDICS INC,1302620.0,2021-01-17,1238291,Fully Received,MassEmail,Caller,CerapedicsAccounts@cerapedics.com,Bill Wilhoit,None,2021-03-21,2021-03-22 20:00:00,2021-01-18 08:22:47,C-296573,30633982,Statement Received,First Round Sent,19272454,Yodit Kahssai,2021-02-10 06:46:17,Done,2021-03-23 06:25:52,5,5,None,31405165,NaN,NaN,NaN


In [80]:
# `Failed Receipt Follow Up` only level in requests.CallerStatus not in activities.Outcome
(
    requests
    .groupBy('CallerStatus')
    .count()
    # .sort(desc('count'))
    .sort('CallerStatus')
    .show(truncate=False)
)

+-------------------------+------+
|CallerStatus             |count |
+-------------------------+------+
|null                     |77624 |
|Failed Receipt Follow Up |194   |
|Left Voicemail           |8026  |
|Need to Receive          |7170  |
|Needs Account Number     |878   |
|Needs Lead               |407   |
|Needs Research           |1874  |
|Needs Supervisor         |4747  |
|Remove From Scope        |17218 |
|Sent Authorization Letter|111548|
|Statement Received       |81877 |
|Verbal Zero Balance      |2498  |
|Will Email or Fax        |1320  |
|Will Not Comply          |30818 |
+-------------------------+------+



In [112]:
# `Partial Receipt` and `Fully Received` of interest
(
    requests
    .groupBy('Status')
    .count()
    # .sort(desc('count'))
    .sort('Status')
    .show(truncate=False)
)

+----------------------+------+
|Status                |count |
+----------------------+------+
|null                  |8     |
|Account Identification|14267 |
|Fully Received        |87644 |
|No Receipt            |52884 |
|Partial Receipt       |3772  |
|Superceded            |187624|
+----------------------+------+



In [116]:
# top value is `Statement Received`
(
    requests
    .where(col('Status').rlike('Recei'))
    .groupBy('CallerStatus')
    .count()
    .sort(desc('count'))
    .show(truncate=False)
)

+-------------------------+-----+
|CallerStatus             |count|
+-------------------------+-----+
|Statement Received       |59392|
|null                     |27524|
|Sent Authorization Letter|20081|
|Remove From Scope        |17032|
|Will Not Comply          |6159 |
|Left Voicemail           |4800 |
|Needs Supervisor         |2432 |
|Need to Receive          |1982 |
|Verbal Zero Balance      |1764 |
|Needs Research           |1569 |
|Will Email or Fax        |645  |
|Needs Account Number     |608  |
|Needs Lead               |221  |
|Failed Receipt Follow Up |91   |
+-------------------------+-----+



In [81]:
(
    activities
    .groupBy('Outcome')
    .count()
    # .sort(desc('count'))
    .sort('Outcome')
    .show(truncate=False)
)

+-------------------------+-------+
|Outcome                  |count  |
+-------------------------+-------+
|null                     |1198265|
|Left Voicemail           |55354  |
|Need to Receive          |8447   |
|Needs Account Number     |1562   |
|Needs Lead               |720    |
|Needs Research           |6035   |
|Needs Supervisor         |7322   |
|Remove From Scope        |5665   |
|Sent Authorization Letter|238424 |
|Statement Received       |69515  |
|Verbal Zero Balance      |2044   |
|Will Email or Fax        |9869   |
|Will Not Comply          |54736  |
+-------------------------+-------+



In [118]:
(
    requests
    .groupBy('RequestMethod')
    .count()
    .sort(desc('count'))
    .show()
)

+-------------+------+
|RequestMethod| count|
+-------------+------+
|    MassEmail|214024|
|          WNC| 56293|
|      MassFax| 30238|
|        AdHoc| 28715|
|        Macro| 12096|
|         Call|  4187|
|    NeedsLead|   391|
|  ClientEmail|   247|
|         null|     8|
+-------------+------+



In [119]:
(
    requests
    .groupBy('RequestType')
    .count()
    .sort(desc('count'))
    .show()
)

+-----------+------+
|RequestType| count|
+-----------+------+
|     Caller|144206|
|     Mass 1|126803|
|       null| 49193|
|     Mass 2| 19845|
|      Macro|  6152|
+-----------+------+



In [130]:
(
    requests
    .where('RequestMethod IS NOT NULL AND RequestType IS NOT NULL')
    .groupBy('RequestMethod', 'RequestType')
    .count()
    # .sort(desc('count'))
    .sort('RequestMethod', 'RequestType')
    .show(42, truncate=False)
)

+-------------+-----------+------+
|RequestMethod|RequestType|count |
+-------------+-----------+------+
|AdHoc        |Caller     |28535 |
|AdHoc        |Macro      |19    |
|Call         |Caller     |3992  |
|Call         |Macro      |129   |
|ClientEmail  |Caller     |244   |
|ClientEmail  |Macro      |1     |
|Macro        |Caller     |4486  |
|Macro        |Macro      |5731  |
|MassEmail    |Caller     |53556 |
|MassEmail    |Mass 1     |113665|
|MassEmail    |Mass 2     |16430 |
|MassFax      |Caller     |9758  |
|MassFax      |Mass 1     |13138 |
|MassFax      |Mass 2     |3415  |
|NeedsLead    |Caller     |391   |
|WNC          |Caller     |43236 |
|WNC          |Macro      |272   |
+-------------+-----------+------+



In [126]:
(
    requests
    # .where(col('Status').rlike('Recei'))
    .groupBy('RequestMethod', 'Status')
    .count()
    .sort(desc('count'))
    # .show(24, truncate=False)
    .count()
)

42

In [ ]:
(
    requests
    .where(col('Status').rlike('Recei'))
    .groupBy('RequestType', 'Status')
    .count()
    .sort(desc('count'))
    .show(24, truncate=False)
)

In [99]:
# RequestMethod, RequestType

columns = [
    'CustomerName',
    'ReferenceNumber',
    'VendorContactObjectID',
    'CreatedDate',
    'ObjectID'
]

request = requests

for column in columns:
    request = request.withColumnRenamed(column, 'Request' + column)

In [117]:
# activities.ActivityDate, requests.LastActivityDate should match? False
# (
#     activities
#     .join(
#         request,
#         on=[
#             activities['StatementRequestObjectID'] == request['RequestObjectID'],
#             activities['Outcome'] != request['CallerStatus'], # 288,382
#             activities['ActivityDate'] == request['LastActivityDate'] # 315,044
#         ],
#         # how='left_semi'
#         how='inner'
#     )
#     # .count()
#     # .select('ActivityDate', 'LastActivityDate')
#     # .show()
#     .limit(10)
#     .toPandas()
# )

In [60]:
keys = [
    'JobNumber',
    'CustomerVendorName',
    'ReferenceNumber'
]

columnFactorCounts(activity, keys, 'StatementRequestObjectID').show(truncate=False)

+---------+------------------------------+---------------+------------------------+
|JobNumber|CustomerVendorName            |ReferenceNumber|StatementRequestObjectID|
+---------+------------------------------+---------------+------------------------+
|3432     |ELSEVIER HEALTH SCIENCE       |1290060        |16                      |
|3432     |ALLEN MEDICAL SYSTEMS INC     |1290366        |16                      |
|3444     |CARDINAL HEALTH 110 LLC       |1293384        |16                      |
|3174     |SMITHS MEDICAL ASD INC        |1231344        |15                      |
|2936     |PARTSSOURCE INC               |1154728        |15                      |
|3444     |STRYKER ORTHOPAEDICS          |1292942        |15                      |
|2936     |STERICYCLE ENVIRONMENTAL SOLUT|1156871        |15                      |
|2775     |STELTER CO                    |1049754        |15                      |
|3177     |PHILIPS MEDICAL SYSTEMS INC   |1225218        |15                

In [65]:
conditions = [
    "JobNumber = 3432",
    "CustomerVendorName = 'ELSEVIER HEALTH SCIENCE'",
    "ReferenceNumber = 1290060"
]

condition = ' AND '.join(conditions)

activity.where(condition).sort('CreatedDate').toPandas()

,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,29052106,2020-11-13 06:35:51,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2020-11-12,None,6214762,Sent 3rd request remote to J.Leavitt@Elsevier.com,None,None,27889072,NaN,29052101,SVC_WORKFLOW
1,29052111,2020-11-13 06:35:52,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2020-11-12,None,6214762,Sent 3rd request remote to J.Leavitt@Elsevier.com,None,None,27889072,NaN,29052107,JFOX
2,29052145,2020-11-13 06:35:55,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2020-11-12,None,6214762,Sent 3rd request remote to J.Leavitt@Elsevier.com,None,None,27889072,NaN,29052140,SVC_WORKFLOW
3,29368796,2020-12-02 03:50:50,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2020-12-01,None,6253686,Started new thread in different format due to ...,None,None,27889072,NaN,29368794,SVC_WORKFLOW
4,29772165,2020-12-18 03:26:01,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2020-12-17,None,6299196,Sent 2nd request remote to J.Leavitt@Elsevier.com,None,None,27889072,NaN,29772163,SVC_WORKFLOW
5,30561511,2021-01-13 03:12:42,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,tkornoelje,2021-01-12,None,6349475,Sent 3rd request remote to J.Leavitt@Elsevier.com,Will Not Comply,None,27889072,NaN,30561508,SVC_WORKFLOW
6,30976645,2021-01-26 03:15:06,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,Tim Kornoelje,2021-01-25,None,6430420,Sent 4th request remote to J.Leavitt@Elsevier.com,Will Not Comply,None,27889072,NaN,30976642,SVC_WORKFLOW
7,31241297,2021-02-09 09:45:05,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,Tim Kornoelje,2021-02-08,None,6465805,Last statement came from jeff.olson@Elsevier.c...,Will Not Comply,None,27889072,NaN,31241288,SVC_WORKFLOW
8,31775265,2021-02-23 06:25:00,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,Tim Kornoelje,2021-02-22,None,6503150,Unable to locate the last statement request. R...,Will Not Comply,None,27889072,31402981.0,31775262,SVC_WORKFLOW
9,32021588,2021-03-09 04:18:02,1290060,ELSEVIER HEALTH SCIENCE,3432,Centra Health - 3432,Client Email,Tim Kornoelje,2021-03-08,None,6534398,Sent 2nd request remote to jeff.olson@Elsevier...,Will Not Comply,None,27889072,31402981.0,32021585,SVC_WORKFLOW


In [68]:
keys = activity.where(condition).select('StatementRequestObjectID').rdd.map(lambda x: x[0]).collect()

In [72]:
# 
requests.where(col('ObjectID').isin(keys)).sort('CreatedDate').toPandas()

,JobNo,JobTier,CustomerName,CustVendorID,VendorNo,CustVendorGroupID,WNC,StatementWNC,VendorGroupName,Volume,RequestDate,ReferenceNumber,Status,RequestMethod,RequestType,Contact,RequesterFullName,RequestText,LastActivityDate,LastStatementReceivedDate,CreatedDate,CallsheetNo,ObjectID,CallerStatus,ReconStatus,CurrentAssigneeID,CurrentAssigneeName,EnteredReconDate,LastReconQueueName,LastReconQueueEntryDate,AccountsReceived,AccountsRequested,MessageID,VendorContactObjectID,WebsiteVendor,WNCSpecialHandling,NeedLeadVendor
0,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2020-11-12,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2020-11-12,NaT,2020-11-13 06:35:51,C-359827,29052101,None,None,16916138,Kermitha Norman,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
1,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2020-11-12,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2020-11-12,NaT,2020-11-13 06:35:51,C-359827,29052107,None,None,16916138,Kermitha Norman,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
2,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2020-11-12,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2020-11-12,NaT,2020-11-13 06:35:55,C-359827,29052140,None,None,16916138,Kermitha Norman,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
3,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2020-12-01,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2020-12-01,NaT,2020-12-02 03:50:46,C-359827,29368794,None,None,16916138,Kermitha Norman,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
4,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2020-12-17,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2020-12-17,NaT,2020-12-18 03:26:01,C-359827,29772163,Will Not Comply,None,16916138,Kermitha Norman,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
5,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2021-01-12,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2021-01-12,NaT,2021-01-13 03:12:41,C-359827,30561508,Will Not Comply,None,8404677,Tim Kornoelje,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
6,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2021-01-25,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2021-01-25,NaT,2021-01-26 03:15:04,C-359827,30976642,Will Not Comply,None,8404677,Tim Kornoelje,NaT,None,NaT,NaN,NaN,None,NaN,0,NaN,NaN
7,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2021-02-08,1290060,Superceded,WNC,Caller,J.Leavitt@Elsevier.com,Tim Kornoelje,None,2021-02-08,NaT,2021-02-09 09:45:02,C-359827,31241288,Will Not Comply,None,8404677,Tim Kornoelje,NaT,None,NaT,NaN,NaN,None,31402981.0,0,NaN,NaN
8,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2021-02-22,1290060,Superceded,WNC,Caller,jeff.olson@Elsevier.com,Tim Kornoelje,None,2021-02-22,NaT,2021-02-23 06:24:57,C-359827,31775262,Will Not Comply,None,8404677,Tim Kornoelje,NaT,None,NaT,NaN,NaN,None,31402981.0,0,NaN,NaN
9,3432,NaN,Centra Health,27889072,33354,27889070,0,1,ELSEVIER HEALTH SCIENCE,123417.0,2021-03-08,1290060,Superceded,WNC,Caller,jeff.olson@Elsevier.com,Tim Kornoelje,None,2021-03-08,NaT,2021-03-09 04:17:59,C-359827,32021585,Will Not Comply,None,8404677,Tim Kornoelje,NaT,None,NaT,0.0,1.0,None,31402981.0,0,NaN,NaN


In [ ]:
spark.stop()